# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [20]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
#!pip install wn
#!python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#%pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\IMOE001\Downloads


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

changing special chars , notes and numbers

In [10]:
from sklearn.metrics import make_scorer, f1_score # make scorer import


def regOnDate(df):
    processed_df = df.copy() # create a new DataFrame with the same shape as the original df
    for i in processed_df.index:
        processed_df.iloc[i,0] = re.sub(r'\W',' ',str(processed_df.iloc[i,0])) 
        processed_df.iloc[i,0] = re.sub(r'\d',' ',str(processed_df.iloc[i,0])) 
        processed_df.iloc[i,0] = re.sub(r'\s\w\s',' ',processed_df.iloc[i,0]) 
    return processed_df['story']

# cleaning the train data and vectorizing

In [27]:

df_train['story'] = df_train['story'].apply(lambda x: x.lower())
df_test['story'] = df_test['story'].apply(lambda x: x.lower())
df_train['story'] = regOnDate(df_train)
df_test['story'] = regOnDate(df_test)
cv = CountVectorizer(token_pattern=r'\b\w+\b', lowercase=False)
x_train = cv.fit_transform(df_train['story'])
x_test = cv.transform(df_test['story'])
le = LabelEncoder()
y_train = le.fit_transform(df_train['gender'])





# training models such as Naive Bayes, SVM, and Neural Networks etc

In [28]:

models = [LinearSVC(),Perceptron(),SGDClassifier(),MultinomialNB(),KNeighborsClassifier(),DecisionTreeClassifier()]
for model in models:
    scores = cross_val_score(model, x_train, y_train, cv=5, scoring=make_scorer(f1_score, average='macro'))
    print(f"{type(model).__name__}: F1-score = {scores.mean():.3f}")

LinearSVC: F1-score = 0.691
Perceptron: F1-score = 0.642
SGDClassifier: F1-score = 0.652
MultinomialNB: F1-score = 0.500
KNeighborsClassifier: F1-score = 0.484
DecisionTreeClassifier: F1-score = 0.588


# hyperparameters of models using GridSearchCV() function,the model with the highest F1-score is saved for last

In [29]:
param_grid = {
    'penalty': [None, 'l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'max_iter': [1000, 2000, 5000],
    'tol': [1e-3, 1e-4, 1e-5],
    'shuffle': [True, False]
}

perceptron = Perceptron()
grid_search = GridSearchCV(perceptron, param_grid, cv=5, scoring=make_scorer(f1_score, average='macro'))
grid_search.fit(x_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best F1-score: {grid_search.best_score_:.3f}")

GridSearchCV(cv=5, estimator=Perceptron(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1],
                         'max_iter': [1000, 2000, 5000],
                         'penalty': [None, 'l1', 'l2', 'elasticnet'],
                         'shuffle': [True, False],
                         'tol': [0.001, 0.0001, 1e-05]},
             scoring=make_scorer(f1_score, average=macro))

Best parameters: {'alpha': 0.0001, 'max_iter': 1000, 'penalty': 'l1', 'shuffle': False, 'tol': 0.001}
Best F1-score: 0.659


In [30]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring=make_scorer(f1_score, average='macro'))
grid_search.fit(x_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best F1-score: {grid_search.best_score_:.3f}")

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             scoring=make_scorer(f1_score, average=macro))

Best parameters: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
Best F1-score: 0.542


In [35]:

param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2','l3']
    
}
svm = LinearSVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring=make_scorer(f1_score, average='macro'))
grid_search.fit(x_train, y_train)
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best F1-score: {grid_search.best_score_:.3f}")



GridSearchCV(cv=5, estimator=LinearSVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'penalty': ['l1', 'l2', 'l3']},
             scoring=make_scorer(f1_score, average=macro))

Best parameters: {'C': 1000, 'penalty': 'l2'}
Best F1-score: 0.700


# final score is 0.7 now all we need to do is predict the gender on the test dataset

In [36]:
# Preprocess the text data by removing stopwords, punctuation, and converting the text to lowercase using CountVectorizer()
new_X = cv.transform(df_test['story'])

# Predict the gender of the stories using the trained SVM model
new_y_pred = grid_search.best_estimator_.predict(new_X)

# Convert the predicted gender labels back to their original values using inverse_transform() method of LabelEncoder()
new_y_pred = le.inverse_transform(new_y_pred)

# Print the predicted gender labels
df_predicted = pd.DataFrame(columns = ['test_example_id','predicted_category'])

df_predicted['test_example_id'] = df_test['test_example_id']
df_predicted['predicted_category'] = new_y_pred
df_predicted

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,f
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [37]:
df_predicted.to_csv('classification_results.csv',index=False)